# **Plotlyを用いたOCTの3D再構成**

In [ ]:
import os
import glob
import re
import numpy as np
import cv2
import plotly.graph_objects as go
from natsort import natsorted
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import cv2

# Colab用レンダラー設定
pio.renderers.default = "colab"

# 画像読み込み（前のコードと同じ）
images = []
for f in sorted_files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img_small = cv2.resize(img, (100, 100))  # さらに小さくして軽量化 (100x100ピクセルにリサイズ)
        images.append(img_small)

volume = np.array(images, dtype=np.float32)
print(f"Volume shape: {volume.shape}")  # (n_slices, height, width)

# 正しい座標グリッドの作成
n_slices, height, width = volume.shape
Z, Y, X = np.mgrid[0:n_slices, 0:height, 0:width]

# 値を正規化（0-255 → 0-1）
vol_normalized = volume / 255.0

# 閾値設定（背景を除外）
threshold = 0.25  # 調整してください

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=vol_normalized.flatten(),
    isomin=threshold,
    isomax=0.9,
    opacity=0.8,
    surface_count=15,
    colorscale='Gray',
    caps=dict(x_show=False, y_show=False, z_show=False),
))

fig.update_layout(
    title="OCT 3D Volume",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z (Slice)",
        aspectmode='data',
    ),
    width=700,
    height=600,
)

fig.show()

Volume shape: (5, 100, 100)


In [ ]:
import os, glob, re
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import cv2

# Colab用レンダラー設定
pio.renderers.default = "colab"

# =========================
# 1) データ場所 & 患者IDを指定
# =========================
base_dir = "/content/drive/MyDrive/nv_detection_project/raw_data/original"

# 表示したい患者ID（例: "10385045"）
# ※ None にすると、見つかった患者IDの先頭を自動選択します
patient_id = None

# さらに軽量化したい場合（例: 100x100）
resize_wh = (100, 100)   # (W, H)

# 閾値など（あなたのコードに合わせる）
threshold = 0.25
isomax = 0.9
opacity = 0.8
surface_count = 15

# =========================
# 2) original直下から patientID-index.tif を集める
# =========================
def parse_patient_and_index(path: str):
    base = os.path.basename(path)
    m = re.match(r"^([A-Za-z0-9]+)-(\d+)\.(tif|tiff)$", base, re.IGNORECASE)
    if not m:
        return None, None
    return m.group(1), int(m.group(2))

all_files = glob.glob(os.path.join(base_dir, "*.tif")) + glob.glob(os.path.join(base_dir, "*.tiff"))
if len(all_files) == 0:
    raise FileNotFoundError(f"No tif/tiff found in: {base_dir}")

# 患者ID一覧を作る
patient_ids = sorted({parse_patient_and_index(f)[0] for f in all_files if parse_patient_and_index(f)[0] is not None})
if len(patient_ids) == 0:
    raise ValueError("No files matched 'patientID-index.tif' pattern (e.g., 10385045-1.tif).")

if patient_id is None:
    patient_id = patient_ids[0]

if patient_id not in patient_ids:
    raise KeyError(f"patient_id='{patient_id}' not found. Example IDs: {patient_ids[:10]}")

# 指定患者のファイルだけ抽出して「連番」でソート
pairs = []
for f in all_files:
    pid, idx = parse_patient_and_index(f)
    if pid == patient_id:
        pairs.append((idx, f))

if len(pairs) == 0:
    raise ValueError(f"No slices found for patient_id='{patient_id}'.")

sorted_files = [f for idx, f in sorted(pairs, key=lambda x: x[0])]

print("Selected patient_id:", patient_id)
print("Num slices:", len(sorted_files))
print("First file:", os.path.basename(sorted_files[0]))
print("Last  file:", os.path.basename(sorted_files[-1]))

# =========================
# 3) 画像読み込み（あなたのコードに準拠）
# =========================
images = []
for f in sorted_files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img_small = cv2.resize(img, resize_wh)  # 軽量化
        images.append(img_small)

volume = np.array(images, dtype=np.float32)
print(f"Volume shape: {volume.shape}")  # (n_slices, height, width)

# 安全チェック
if volume.ndim != 3 or volume.shape[0] < 2:
    raise ValueError("Volume is not valid (need 3D array with multiple slices).")

# =========================
# 4) 座標グリッド作成（あなたのコードに準拠）
# =========================
n_slices, height, width = volume.shape
Z, Y, X = np.mgrid[0:n_slices, 0:height, 0:width]

# =========================
# 5) 正規化（あなたのコードに準拠）
# =========================
# ※ tifが必ずしも0-255とは限らないので、min-max正規化の方が安全
vmin, vmax = float(volume.min()), float(volume.max())
vol_normalized = (volume - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(volume)

# =========================
# 6) Plotly Volume（あなたのコードに準拠）
# =========================
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=vol_normalized.flatten(),
    isomin=threshold,
    isomax=isomax,
    opacity=opacity,
    surface_count=surface_count,
    colorscale='Gray',
    caps=dict(x_show=False, y_show=False, z_show=False),
))

fig.update_layout(
    title=f"OCT 3D Volume (patient_id={patient_id})",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z (Slice)",
        aspectmode='data',
    ),
    width=700,
    height=600,
)

fig.show()


Selected patient_id: 10385045L
Num slices: 20
First file: 10385045L-1.tif
Last  file: 10385045L-20.tif
Volume shape: (20, 100, 100)
